<a href="https://colab.research.google.com/github/ClaireZixiWang/SimCLR/blob/master/cifar10_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Accuracy of the MTL model on the 10000 test images: 74 %

# Base Resnet18

## Train base resnet-18 model with same hyper parameter

### Data Loader

In [1]:


import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms 
import torchvision.models as models
import tqdm


In [2]:



# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 100
learning_rate = 0.0003
BATCH_SIZE = 256
weight_decay = 1e-4
PATH = '/content/drive/MyDrive/resnet18'

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=BATCH_SIZE, 
                                          shuffle=False)



  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../../data/cifar-10-python.tar.gz to ../../data/


In [ ]:
device

device(type='cuda')

### Training

In [ ]:

model = models.resnet18(pretrained=False)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate, weight_decay=weight_decay)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader), eta_min=0,
                                                           last_epoch=-1)

# For updating learning rate

# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
      print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

  # Decay learning rate
  if epoch >= 10:
    scheduler.step()                                                           

  if epoch %10 == 0:
    checkpoint_name = 'checkpoint_resnet18-base_{:04d}.pth.tar'.format(epoch)
    torch.save(model.state_dict(), PATH+'/'+checkpoint_name)

checkpoint_name = 'checkpoint_resnet18-base{:04d}.pth.tar'.format(num_epochs)
torch.save(model.state_dict(), PATH+'/'+checkpoint_name)


## Loading the model and Attack

### Load the model

In [3]:
PATH = '/content/drive/MyDrive/resnet18/checkpoint_resnet18-base_0090.pth.tar'
model_state_dict = torch.load(PATH)


In [4]:
model = models.resnet18(pretrained=False)
model.load_state_dict(model_state_dict)
model = model.to(device)


### PGD attack the model and save images

In [5]:
!pip install torchattacks

     |████████████████████████████████| 102 kB 3.4 MB/s 


In [8]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms

# import torchattacks
from torchattacks import PGD, FGSM

# from models import Holdout, Target
# from utils import imshow

In [7]:
batch_size = 24

cifar10_train = dsets.CIFAR10(root='./data', train=True,
                              download=True, transform=transforms.ToTensor())
cifar10_test  = dsets.CIFAR10(root='./data', train=False,
                              download=True, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(cifar10_train,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(cifar10_test,
                                          batch_size=batch_size,
                                          shuffle=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


#### FGSM

In [9]:
atk = FGSM(model, eps=8/255)
# atk.set_return_type('int') # Save as integer.
atk.save(data_loader=test_loader, save_path="/content/drive/MyDrive/mtlsimclr/cifar10_fgsm.pt", verbose=True)

- Save progress: 100.00 % / Robust accuracy: 2.89 % / L2: 1.72355 (0.013 it/s) 	
- Save complete!


#### PGD-7

In [16]:
atk = PGD(model, eps=8/255, alpha=2/255, steps=7)
# atk.set_return_type('int') # Save as integer.
atk.save(data_loader=test_loader, save_path="/content/drive/MyDrive/mtlsimclr/cifar10_pgd.pt", verbose=True)

- Save progress: 100.00 % / Robust accuracy: 0.04 % / L2: 1.37845 (0.059 it/s) 	
- Save complete!


#### PGD-20

In [17]:
atk = PGD(model, eps=8/255, alpha=2/255, steps=20)
# atk.set_return_type('int') # Save as integer.
atk.save(data_loader=test_loader, save_path="/content/drive/MyDrive/mtlsimclr/cifar10_pgd_20.pt", verbose=True)

- Save progress: 100.00 % / Robust accuracy: 0.00 % / L2: 1.51923 (0.161 it/s) 	
- Save complete!


#### PGD-50

In [18]:
atk = PGD(model, eps=8/255, alpha=2/255, steps=50)
# atk.set_return_type('int') # Save as integer.
atk.save(data_loader=test_loader, save_path="/content/drive/MyDrive/mtlsimclr/cifar10_pgd_50.pt", verbose=True)

- Save progress: 100.00 % / Robust accuracy: 0.00 % / L2: 1.55687 (0.399 it/s) 	
- Save complete!


### Natural Accuracy


In [19]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'resnet.ckpt')


Accuracy of the model on the test images: 82.46 %


### Robust Accuracy

#### FGSM

In [20]:
adv_images, adv_labels = torch.load("/content/drive/MyDrive/mtlsimclr/cifar10_fgsm.pt")
adv_data = TensorDataset(adv_images, adv_labels)
adv_loader_fgsm = DataLoader(adv_data, batch_size=128, shuffle=False)

In [21]:
model.eval()

correct = 0
total = 0

for images, labels in adv_loader_fgsm:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 2.89 %


#### PGD-7

In [22]:
adv_images, adv_labels = torch.load("/content/drive/MyDrive/mtlsimclr/cifar10_pgd.pt")
adv_data = TensorDataset(adv_images, adv_labels)
adv_loader_7 = DataLoader(adv_data, batch_size=128, shuffle=False)

In [23]:
model.eval()

correct = 0
total = 0

for images, labels in adv_loader_7:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 0.04 %


#### PGD-20

In [24]:
adv_images, adv_labels = torch.load("/content/drive/MyDrive/mtlsimclr/cifar10_pgd_20.pt")
adv_data = TensorDataset(adv_images, adv_labels)
adv_loader_20 = DataLoader(adv_data, batch_size=128, shuffle=False)

In [25]:
model.eval()

correct = 0
total = 0

for images, labels in adv_loader_20:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 0.00 %


#### PGD-50

In [26]:
adv_images, adv_labels = torch.load("/content/drive/MyDrive/mtlsimclr/cifar10_pgd_50.pt")
adv_data = TensorDataset(adv_images, adv_labels)
adv_loader_50 = DataLoader(adv_data, batch_size=128, shuffle=False)

In [27]:
model.eval()

correct = 0
total = 0

for images, labels in adv_loader_50:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 0.00 %
